<a href="https://colab.research.google.com/github/midhun-james/gliner-desc/blob/main/rag_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai langchain-google-genai langchain-community chromadb sentence-transformers

In [2]:
pip install -q pymupdf

In [3]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ''
os.environ['NO_GCE_CHECK'] = 'True'

Setting up the api key configuration

In [29]:
from google.colab import userdata
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_retries=2
)


In [82]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
import pprint
import fitz
import pandas as pd

Copied the text in 10-k reports to a txt file.

Used fitz package from Pymupdf to extract text from the pdf

In [47]:

pdf_path='assign_3_data.txt'
doc=fitz.open(pdf_path)
text=''
for page in doc:
  text+=page.get_text()
print(text[:500])

 
 
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Transition Period From                  to
Commission File Number 001-37845
 
MICROSOFT CORPORATION
 
 
Washington
 
91-1144442
(STATE OF INCORPORATION)
 
(I


For text splitting setted the chunk size as 1500 and chunk overlap as 200.

Since it is a very large file chunk making chunk size bigger is needed. overlapping is setted inorder to not loose continuity

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks=text_splitter.split_text(text)



In [15]:
print(f'Total chunks: {len(chunks)}')
print(f'first chunk : {chunks[0]}')

Total chunks: 286
first chunk : UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Transition Period From                  to
Commission File Number 001-37845
 
MICROSOFT CORPORATION
 
 
Washington
 
91-1144442
(STATE OF INCORPORATION)
 
(I.R.S. ID)
 
ONE MICROSOFT WAY, REDMOND, Washington 98052-6399
(425) 882-8080
www.microsoft.com/investor
 
 
 
 
 
Securities registered pursuant to Section 12(b) of
the Act:
 
 
 
 
 
 
 
 
 
Title of each class
 
Trading Symbol
 
Name of exchange on which registered
 
 
 
 
 
Common stock, $0.00000625 par value per share
 
MSFT
 
Nasdaq
3.125% Notes due 2028
 
MSFT
 
Nasdaq
2.625% Notes due 2033
 
MSFT
 
Nasdaq
 
 
 
 
 
Securities registered pursuant to Section 12(g

For RAG we need these text chunks as documents.

By doing this we can get a metadata along with the text content

In [16]:
docs = text_splitter.create_documents([text])
print(f"Total documents: {len(docs)}")
print(docs[0].page_content[:300])

Total documents: 286
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 10-K
 
 
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE
SECURITIES EXCHANGE ACT OF 1934
 
 
 
For the Fiscal Year Ended June 30, 2022
 
 
 
OR
 
 
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF
THE SECURIT


In [17]:
embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



/tmp/ipython-input-3551722535.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access p

In [18]:
# Create a Chroma vector store and persist it locally
db = Chroma.from_documents(documents=docs, embedding=embedding_function, persist_directory="chroma_store")

# Save to disk
db.persist()

/tmp/ipython-input-2790807997.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


Set {"k": 10} which means the retriever picks up 3 chunks or documents

In [50]:
persist_directory = "chroma_store"

embedding_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)

retriever = vector_store.as_retriever(search_kwargs={"k": 10})

QA chain is defined where chain type is set as 'stuff' which is a simple mode whch will stuff all the retrived chunks into a single prompt

In [51]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [52]:
query = "What were the company's total revenues for the fiscal year that ended on June 30, 2022?"
result = qa_chain.invoke(query)


In [53]:
result['result']

"The company's total revenues for the fiscal year that ended on June 30, 2022, were $198,270 million."

By reviwing the source document we can see the value **$198,270** there hence we know the answer is correct

In [54]:
result['source_documents'][0].page_content

'services, and customer service and support. Each\nallocation is measured differently based on the\nspecific facts and circumstances of the costs being\nallocated.\nSegment revenue and operating income were as follows\nduring the periods presented:\n \n(In millions)\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nYear Ended June 30,\n \n \n2022\n \n \n \n2021\n \n \n \n2020\n \n \n \n \n \n \n \n \n \n \n \n \n \n \nRevenue\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nProductivity and Business Processes\n \n$\n63,364\n \n \n$\n53,915\n \n \n$\n46,398\n \nIntelligent Cloud\n \n \n75,251\n \n \n \n60,080\n \n \n \n48,366\n \nMore Personal Computing\n \n \n59,655\n \n \n \n54,093\n \n \n \n48,251\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nTotal\n \n$\n198,270\n \n \n$\n168,088\n \n \n$\n143,015\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nOperating Income\n \n \n \n \n \n \n \n \n \n \n \n \

Creating a dataframe inorder to compare

In [101]:
df=pd.DataFrame(columns=['Question','Answer_before_rerank',])
df_after_rerank=pd.DataFrame(columns=['Question','Answer_after_rerank',])

**Specific fact evaluation**

By analyzing these values we can say it gives very good results

In [110]:
specific_questions= ["list all the directors ","How much did Microsoft spend on research and development during the fiscal year 2022?","Total Number shares purchased from April 1st 2022 to June 30 2022?"]
for query in specific_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][0].page_content}\n\n')
  # Use .loc to add a new row to the DataFrame
  df.loc[len(df)] = {'Question': query, 'Answer_before_rerank': result["result"]}

Question :  list all the directors 



Answer : Here are all the directors listed in the provided text:

*   Reid Hoffman
*   Hugh F. Johnston
*   Teri L. List
*   Sandra E. Peterson
*   Penny S. Pritzker
*   Carlos A. Rodriguez
*   Charles W. Scharf
*   John W. Stanton
*   John W. Thompson (Lead Independent Director)
*   Emma N. Walmsley
*   Padmasree Warrior


('Source Document : Reid Hoffman\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ HUGH F. JOHNSTON        \n'
 ' \n'
 'Hugh F. Johnston\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ TERI L. LIST\n'
 ' \n'
 'Teri L. List\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ SANDRA E. PETERSON\n'
 ' \n'
 'Sandra E. Peterson\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 ' \n'
 '/s/ PENNY S. PRITZKER\n'
 ' \n'
 'Penny S. Pritzker\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ CARLOS A. RODRIGUEZ\n'
 ' \n'
 'Director\n'
 'Carlos A. Rodriguez\n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 '/s/ CHARLES W. SCHARF        \n'
 ' \n'
 'Charles W. Scharf\n'
 ' \n'
 'Director\n'
 '

Summarization questions


In [111]:
summarization_questions=["Summarize the major risks that Microsoft identifies related to cybersecurity and data privacy.","What ongoing legal or regulatory investigations does Microsoft disclose?"]

for query in summarization_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][0].page_content}\n\n')
  df.loc[len(df)] = {'Question': query, 'Answer_before_rerank': result["result"]}


Question :  Summarize the major risks that Microsoft identifies related to cybersecurity and data privacy.



Answer : Microsoft identifies several major risks related to cybersecurity and data privacy:

*   **Diverse Threat Actors:** Attacks originate from individual and groups of hackers, sophisticated organizations, state-sponsored organizations, and nation-states. These nation-state attacks can be particularly resource-intensive and may intensify during certain periods.
*   **Variety of Attack Methods:** These actors employ a wide range of methods, including developing and deploying malicious software, exploiting vulnerabilities in hardware, software, or other infrastructure, using social engineering techniques to obtain sensitive information (like passwords), and launching distributed denial of service (DDoS) or other coordinated attacks.
*   **Broad Impact:** Successful cyberattacks and security vulnerabilities can lead to significant negative consequences, such as reduced revenu

Keyword Dependendent questions

In [112]:
keyword_questions=["What does the report mention about 'Windows OEM' revenue trends?","What does the report state about Microsoft’s 'gaming' or 'Xbox' business performance in fiscal year 2022?"]

for query in keyword_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][0].page_content}\n\n')
  df.loc[len(df)] = {'Question': query, 'Answer_before_rerank': result["result"]}

Question :  What does the report mention about 'Windows OEM' revenue trends?



Answer : The report states that Windows OEM revenue increased by 11%. This growth was driven by continued strength in the commercial PC market, which has a higher revenue per license.


('Source Document : business within this segment. These metrics provide\n'
 'strategic product insights which allow us to assess\n'
 'the performance across our commercial and consumer\n'
 'businesses. As we have diversity of target audiences\n'
 'and sales motions within the Windows business, we\n'
 'monitor metrics that are reflective of those varying\n'
 'motions.\n'
 ' \n'
 'Windows OEM revenue growth\n'
 ' \n'
 'Revenue from sales of Windows Pro and non-Pro\n'
 'licenses sold through the OEM channel\n'
 ' \n'
 ' \n'
 ' \n'
 'Windows Commercial products and cloud services\n'
 'revenue growth\n'
 ' \n'
 'Revenue from Windows Commercial products and cloud\n'
 'services, comprising volume licensing of the Windows\n'
 'opera

**Advcanced RAG**

In [58]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [62]:
def rerank_documents(documents, query, cross_encoder, top_n=3):
    pairs = [[query, doc.page_content] for doc in documents]
    scores = cross_encoder.predict(pairs)
    doc_score_pairs = list(zip(documents, scores))
    doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
    return [doc for doc, score in doc_score_pairs[:top_n]]
def retrieve_and_rerank(vector_store, query, cross_encoder, k=10, top_n=3):
    docs = vector_store.similarity_search(query, k=k)
    reranked_docs = rerank_documents(docs, query, cross_encoder, top_n=top_n)
    return reranked_docs

In [66]:
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.schema.retriever import BaseRetriever
from typing import List
from pydantic import Field

class RerankRetriever(BaseRetriever):
    vector_store: any = Field(...)
    cross_encoder: any = Field(...)
    k: int = Field(default=10)
    top_n: int = Field(default=3)

    def _get_relevant_documents(self, query: str) -> List[Document]:
        docs = self.vector_store.similarity_search(query, k=self.k)
        if not docs:
            return []
        pairs = [[query, doc.page_content] for doc in docs]
        scores = self.cross_encoder.predict(pairs)
        doc_score_pairs = list(zip(docs, scores))
        doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
        top_docs = [doc for doc, _ in doc_score_pairs[:self.top_n]]
        return top_docs


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:628: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [68]:

rerank_retriever = RerankRetriever(
    vector_store=vector_store,
    cross_encoder=cross_encoder,
    k=10,
    top_n=3
)

qa_chain_reranked = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=rerank_retriever,
    return_source_documents=True
)


In [102]:
for query in specific_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain_reranked.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][2].page_content}\n\n')
  df_rerank.loc[len(df)] = {'Question': query, 'Answer_after_rerank': result["result"]}

Question :  list all the directors 



Answer : Here are all the directors listed:

*   Reid Hoffman
*   Hugh F. Johnston
*   Teri L. List
*   Sandra E. Peterson
*   Penny S. Pritzker
*   Carlos A. Rodriguez
*   Charles W. Scharf
*   John W. Stanton
*   John W. Thompson (Lead Independent Director)
*   Emma N. Walmsley
*   Padmasree Warrior


('Source Document : Reid Hoffman\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ HUGH F. JOHNSTON        \n'
 ' \n'
 'Hugh F. Johnston\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ TERI L. LIST\n'
 ' \n'
 'Teri L. List\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ SANDRA E. PETERSON\n'
 ' \n'
 'Sandra E. Peterson\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 ' \n'
 '/s/ PENNY S. PRITZKER\n'
 ' \n'
 'Penny S. Pritzker\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ CARLOS A. RODRIGUEZ\n'
 ' \n'
 'Director\n'
 'Carlos A. Rodriguez\n'
 ' \n'
 ' \n'
 ' \n'
 ' \n'
 '/s/ CHARLES W. SCHARF        \n'
 ' \n'
 'Charles W. Scharf\n'
 ' \n'
 'Director\n'
 ' \n'
 ' \n'
 '/s/ JOH

In [103]:
for query in summarization_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain_reranked.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][2].page_content}\n\n')
  df_rerank.loc[len(df)] = {'Question': query, 'Answer_after_rerank': result["result"]}

Question :  Summarize the major risks that Microsoft identifies related to cybersecurity and data privacy.



Answer : Microsoft identifies several major risks related to cybersecurity and data privacy, which could lead to reduced revenue, increased costs, liability claims, or harm to their reputation or competitive position.

The threats to their information technology security come from various actors, including:
*   Individual and groups of hackers.
*   Sophisticated organizations, including state-sponsored organizations or nation-states.

These actors employ a wide variety of methods, such as:
*   Developing and deploying malicious software.
*   Exploiting vulnerabilities in hardware, software, or other infrastructure to attack products and services or gain access to networks and datacenters.
*   Using social engineering techniques to trick employees, users, partners, or customers into disclosing passwords or other sensitive information, or taking actions that grant access to data.

In [104]:
for query in keyword_questions:
  print(f'Question :  {query}\n\n\n')
  result = qa_chain_reranked.invoke(query)
  print(f'Answer : {result["result"]}\n\n')
  pprint.pprint(f'Source Document : {result["source_documents"][2].page_content}\n\n')
  df_rerank.loc[len(df)] = {'Question': query, 'Answer_after_rerank': result["result"]}

Question :  What does the report mention about 'Windows OEM' revenue trends?



Answer : The report does not explicitly mention "trends" in Windows OEM revenue (e.g., whether it's increasing or decreasing over time). Instead, it details the various factors that **impact** or **influence** Windows OEM revenue.

These factors include:
*   The number of Windows operating system licenses purchased by OEMs for pre-installation on devices.
*   Computing device market volume.
*   The mix of computing devices based on form factor and screen size.
*   Differences in device market demand between developed and growth markets.
*   Attachment of Windows to devices shipped.
*   Customer mix (consumer, small and medium businesses, large enterprises).
*   Changes in inventory levels in the OEM channel.
*   Pricing changes and promotions, pricing variation due to shifts in device manufacturing (local/regional vs. multinational OEMs), and different pricing of Windows versions.
*   Constraints in the sup

In [113]:
df_combined = df.set_index('Question').join(df_after_rerank.set_index('Question'))
display(df_combined)

,Answer_before_rerank,Answer_after_rerank
Question,,
list all the directors,Here are all the directors listed in the provi...,Here are all the directors listed:\n\n* Reid...
How much did Microsoft spend on research and development during the fiscal year 2022?,"Microsoft spent $24,512 million on research an...",The provided text states that Research and dev...
Total Number shares purchased from April 1st 2022 to June 30 2022?,"From April 1, 2022, to June 30, 2022, a total ...","From April 1, 2022, to June 30, 2022, the tota..."
Summarize the major risks that Microsoft identifies related to cybersecurity and data privacy.,Microsoft identifies several major risks relat...,Microsoft identifies several major risks relat...
What ongoing legal or regulatory investigations does Microsoft disclose?,"Based on the provided context, Microsoft discl...",Microsoft discloses the following ongoing lega...
What does the report mention about 'Windows OEM' revenue trends?,The report states that Windows OEM revenue inc...,"The report does not explicitly mention ""trends..."
What does the report state about Microsoft’s 'gaming' or 'Xbox' business performance in fiscal year 2022?,"In fiscal year 2022, Microsoft's gaming busine...","The report defines ""Xbox content and services ..."




*   The change that specifically seen in the reranking method is that it is giving more grounded results
*   For example in keyword searching before reranking even a word in the question is not in the document it was giving some output , but after reranking it specifically mentions the word is not in the retrieved documents.

